# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate(压缩) them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)
print(len(train_dataset))
print(len(test_dataset))
# encapsulate them into dataloader form
# 可以多线程加速读取数据
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=10)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True, num_workers=10)

60000
10000


In [3]:
feature, label = train_dataset[0]
print(feature.shape, label)
# feature的尺寸是channel*height*width
# 其中channel是1，因为数据集是灰度图像

torch.Size([1, 28, 28]) 5


In [4]:
start = time.time()
for X, y in train_loader:
    continue
print('%.2f sec' % (time.time() - start))

6.46 sec


Then, we define the model, object function and optimizer that we use to classify.

In [5]:
# 网络的输入输出：784个像素，10个概率值
num_inputs = 784
num_outputs = 10

class SimpleNet(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.linear = nn.Linear(num_inputs, 256)# 输入输出变量的数目
        self.linear2 = nn.Linear(256, num_outputs)# 输入输出变量的数目
        
    # 前向传播
    def forward(self, x):
        # 为前⾯我们数据返回的每个batch样本 x 的形状为(batch_size, 1, 28, 28)
        # 所以我们要先⽤ view() 将 x 的形状转换成(batch_size, 784)才送⼊全连接层
        y = self.linear(x.view(x.shape[0], -1))
        y = self.linear2(y)
        return y;
    
# 模型对象    
model = SimpleNet(num_inputs, num_outputs)
# 初始化参数
nn.init.normal_(model.linear.weight, mean=0, std=0.01)
nn.init.constant_(model.linear.bias, val=0)

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [6]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in tqdm(data_iter):
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

Next, we can start to train and evaluate!

In [8]:
# train and evaluate
start_time = time.time()
for epoch in range(NUM_EPOCHS):
    train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images = torch.autograd.Variable(images)
        labels = torch.autograd.Variable(labels)
        labels_hat = model(images)
        loss = criterion(labels_hat, labels).sum()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_l_sum += loss.item()
        train_acc_sum += (labels_hat.argmax(dim=1) == labels).sum().item()
        n += labels.shape[0]  
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    test_acc = evaluate_accuracy(test_loader, model)  
    print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
          % (epoch + 1, train_l_sum / n, train_acc_sum / n,
             test_acc))
    
print('time %.2f'%time.time() - start_time)


  0%|                                                                                          | 0/468 [00:00<?, ?it/s]

epoch 1, loss 0.0040, train acc 0.859, test acc 0.903


  0%|                                                                                          | 0/468 [00:00<?, ?it/s]

epoch 2, loss 0.0026, train acc 0.903, test acc 0.912


  0%|                                                                                          | 0/468 [00:00<?, ?it/s]

...

  0%|                                                                                          | 0/468 [00:00<?, ?it/s]

epoch 9, loss 0.0022, train acc 0.919, test acc 0.918


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 40.29it/s]

epoch 10, loss 0.0022, train acc 0.920, test acc 0.918


TypeError: unsupported operand type(s) for -: 'str' and 'float'

#### Q5:
Please print the training and testing accuracy.

In [10]:
print('loss %.4f, train acc %.3f, test acc %.3f'
          % (train_l_sum / n, train_acc_sum / n,
             test_acc))

loss 0.0022, train acc 0.920, test acc 0.918
